In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import precision_score, recall_score, accuracy_score
import matplotlib.pyplot as plt
import gc

def create_data_generator(images, labels, batch_size=16):
    num_samples = len(images)
    while True:
        indices = np.random.permutation(num_samples)
        for start_idx in range(0, num_samples, batch_size):
            end_idx = min(start_idx + batch_size, num_samples)
            batch_indices = indices[start_idx:end_idx]

            batch_images = images[batch_indices]
            batch_images = np.stack((batch_images,) * 3, axis=-1)
            batch_images = tf.image.resize(batch_images, (96, 96))
            batch_images = batch_images / 255.0

            batch_labels = to_categorical(labels[batch_indices], 10)
            yield batch_images, batch_labels

def build_model(input_shape=(96, 96, 3)):
    base_model = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape,
        pooling='avg'
    )
    base_model.trainable = False

    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.RandomRotation(0.1),
        base_model,
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])

    return model

def evaluate_batch(model, images, labels, batch_size=16):
    total_correct = 0
    total_samples = 0
    predictions = []
    true_labels = []

    for i in range(0, len(images), batch_size):
        batch_images = images[i:i + batch_size]
        batch_labels = labels[i:i + batch_size]

        batch_images = np.stack((batch_images,) * 3, axis=-1)
        batch_images = tf.image.resize(batch_images, (96, 96))
        batch_images = batch_images / 255.0

        batch_preds = model.predict(batch_images, verbose=0)
        batch_preds = np.argmax(batch_preds, axis=1)

        predictions.extend(batch_preds)
        true_labels.extend(batch_labels)

        gc.collect()
        tf.keras.backend.clear_session()

    precision = precision_score(true_labels, predictions, average='weighted')
    recall = recall_score(true_labels, predictions, average='weighted')
    accuracy = accuracy_score(true_labels, predictions)

    return precision, recall, accuracy

def main():
    (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

    train_gen = create_data_generator(train_images, train_labels)
    test_gen = create_data_generator(test_images, test_labels)

    model = build_model()
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    callbacks = [
        EarlyStopping(patience=2, restore_best_weights=True),
        ReduceLROnPlateau(factor=0.2, patience=1, min_lr=1e-6)
    ]

    history = model.fit(
        train_gen,
        steps_per_epoch=len(train_images) // 16,
        epochs=10,
        validation_data=test_gen,
        validation_steps=len(test_images) // 16,
        callbacks=callbacks,
        verbose=1
    )

    print("\nEvaluating model...")
    precision, recall, accuracy = evaluate_batch(model, test_images, test_labels)

    print(f"\nTest Metrics:")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Accuracy: {accuracy:.4f}")

    model.save_weights('fashion_mnist_memory_efficient.weights.h5')

    return model, history

if __name__ == "__main__":
    model, history = main()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 483s 127ms/step - accuracy: 0.7068 - loss: 0.8471 - val_accuracy: 0.8425 - val_loss: 0.4342 - learning_rate: 0.0010

Evaluating model...

Test Metrics:
Precision: 0.8462
Recall: 0.8425
Accuracy: 0.8425


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import precision_score, recall_score, accuracy_score
import matplotlib.pyplot as plt
import gc

physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        for device in physical_devices:
            tf.config.experimental.set_memory_growth(device, True)
        print("GPU memory growth enabled")
    except RuntimeError as e:
        print(f"Error configuring GPU: {e}")
else:
    print("No GPU devices found")

tf.config.optimizer.set_jit(True)
AUTOTUNE = tf.data.experimental.AUTOTUNE



GPU memory growth enabled


In [ ]:
def create_dataset(images, labels, batch_size=32):
    """Create optimized tf.data.Dataset for GPU"""

    images = tf.cast(images, tf.float32)

    def preprocess(image, label):

        image = tf.stack([image, image, image], axis=-1)
        image = tf.image.resize(image, (96, 96))
        image = image / 255.0
        return image, tf.one_hot(label, 10)

    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    dataset = dataset.map(preprocess, num_parallel_calls=AUTOTUNE)
    dataset = dataset.cache()
    dataset = dataset.shuffle(5000)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(AUTOTUNE)

    return dataset


In [ ]:

def build_model(input_shape=(96, 96, 3)):
    with tf.device('/GPU:0'):
        base_model = MobileNetV2(
            weights='imagenet',
            include_top=False,
            input_shape=input_shape,
            pooling='avg'
        )
        base_model.trainable = False

        model = models.Sequential([
            layers.Input(shape=input_shape),
            layers.RandomRotation(0.1),
            base_model,
            layers.Dense(64, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(10, activation='softmax')
        ])

    return model

In [ ]:
@tf.function
def train_step(model, optimizer, loss_fn, images, labels):
    """Custom training step optimized for GPU"""
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_fn(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss, predictions



In [ ]:
def evaluate_model(model, dataset):
    """Evaluate model using GPU-optimized batches"""
    predictions = []
    true_labels = []

    for batch_images, batch_labels in dataset:
        batch_preds = model(batch_images, training=False)
        predictions.extend(tf.argmax(batch_preds, axis=1).numpy())
        true_labels.extend(tf.argmax(batch_labels, axis=1).numpy())

    precision = precision_score(true_labels, predictions, average='weighted')
    recall = recall_score(true_labels, predictions, average='weighted')
    accuracy = accuracy_score(true_labels, predictions)

    return precision, recall, accuracy

def main():
       (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

    batch_size = 64
    train_dataset = create_dataset(train_images, train_labels, batch_size)
    test_dataset = create_dataset(test_images, test_labels, batch_size)

    strategy = tf.distribute.MirroredStrategy()
    with strategy.scope():
        model = build_model()
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

    callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=2,
            min_lr=1e-6,
            verbose=1
        )
    ]

    print("\nTraining model on GPU...")
    history = model.fit(
        train_dataset,
        epochs=10,
        validation_data=test_dataset,
        callbacks=callbacks,
        verbose=1
    )

    print("\nEvaluating model...")
    precision, recall, accuracy = evaluate_model(model, test_dataset)

    print(f"\nTest Metrics:")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Accuracy: {accuracy:.4f}")

    model.save_weights('fashion_mnist_gpu.weights.h5')

    return model, history

if __name__ == "__main__":
    model, history = main()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

Training model on GPU...
Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 47s 38ms/step - accuracy: 0.6729 - loss: 0.9641 - val_accuracy: 0.8462 - val_loss: 0.4195 - learning_rate: 0.0010
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 23s 24ms/step - accuracy: 0.8115 - loss: 0.5339 - val_accuracy: 0.8506 - val_loss: 0.4092 - learning_rate: 0.0010
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - accuracy: 0.8249 - loss: 0.4954 - val_accuracy: 0.8625 - val_loss: 0.3754 - learning_rate: 0.0010
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 16s 17ms/step - accuracy: 0.8340 - loss: 0.4702 - val_accuracy: 0.8650 - val_loss: 0.3646 - learning_rate: 0.0010
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - accuracy: 0.8372 - loss: 0.4581 - val_accuracy: 0.8698 - val_loss